Notebook para procesar las bases del saber pro con la actualizacion del 31/07/2025

In [1]:
import os
import glob

import pandas as pd

import constants as constants
from unidecode import unidecode

import csv

In [2]:
def resumen_nans_df(df):
    """
    Devuelve un DataFrame con el número de NaNs, número de no-NaNs y el porcentaje de NaNs por columna,
    ordenado de mayor a menor porcentaje de NaNs.
    """
    total_filas = df.shape[0]
    nans_por_columna = df.isna().sum()
    no_nans = total_filas - nans_por_columna
    porcentaje_nans = (nans_por_columna / total_filas) * 100

    resumen = pd.DataFrame({
        'n_nans': nans_por_columna.astype(float),
        'n_no_nans': no_nans.astype(float),
        'porcentaje_nans': porcentaje_nans.astype(float)
    })

    resumen = resumen.sort_values(by='porcentaje_nans', ascending=False)
    return resumen

### Procesamiento bases

In [4]:
# Ruta del directorio con los archivos
data_dir_saberpro = "../../data/SABERPRO_raw_reduced/actualizacion_31072025/"
csv_files_saberpro = glob.glob(os.path.join(data_dir_saberpro, "*.txt"))
# Normaliza la ruta antes de remover
#archivo_a_remover = os.path.normpath('../../data/SABERPRO_raw_reduced/saberpro_20232.txt')
csv_files_saberpro = [os.path.normpath(f) for f in csv_files_saberpro]


In [55]:
# Lista para guardar los dataframes
data_list = []

# Leer los archivos del saber pro antes del 2023-2
for file_path in csv_files_saberpro:
    try:
        # Leer sin usecols para evitar errores si faltan columnas
        data_temp = pd.read_csv(
            file_path,
            sep=";",
            engine="python",
            encoding="utf-8"
        )

        # Asegurar que todas las columnas esperadas existan
        for col in constants.cols_saberpro_lower:
            if col not in data_temp.columns:
                data_temp[col] = pd.NA

        # Filtrar solo las columnas deseadas
        data_temp = data_temp[constants.cols_saberpro_lower]

        # Castear columna si existe
        if "estu_cod_mcpio_presentacion" in data_temp.columns:
            data_temp["estu_cod_mcpio_presentacion"] = data_temp["estu_cod_mcpio_presentacion"].astype("Int64")

        data_list.append(data_temp)

    except Exception as err:
        print(f"Error reading {file_path}: {err}")

# Concatenar los dataframes
sbpro = pd.concat(data_list, ignore_index=True)

# Filtrar por ubicación del programa educativo (comentado según instrucciones)
# sbpro = sbpro[sbpro["estu_prgm_codmunicipio"] == COD_MCIPIO_BOGOTA]
# sbpro = sbpro[sbpro["estu_prgm_codmunicipio"].isin(municipios["codigo_dane_municipio"])]

# Ordenar por periodo y resetear índice
sbpro = sbpro.sort_values(by="periodo").reset_index(drop=True)

# Reordenar columnas según orden esperado
sbpro = sbpro[constants.cols_saberpro_lower]

# Liberar memoria
del data_list
del data_temp


C:\Users\Max\AppData\Local\Temp\ipykernel_18656\4148166615.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sbpro = pd.concat(data_list, ignore_index=True)


In [56]:
#parse las columnas a los tipos de variables adecuados
for col, dtype in constants.dtype_mapping_saberpro.items():
    if col in sbpro.columns:
        try:
            sbpro[col] = sbpro[col].astype(dtype)
        except Exception as e:
            print(f"No se pudo convertir la columna {col} a {dtype}: {e}")

No se pudo convertir la columna punt_global a int64: Cannot convert non-finite values (NA or inf) to integer
No se pudo convertir la columna estu_prgm_codmunicipio a int64: Cannot convert non-finite values (NA or inf) to integer
No se pudo convertir la columna inst_cod_institucion a int64: Cannot convert non-finite values (NA or inf) to integer
No se pudo convertir la columna estu_inst_codmunicipio a int64: Cannot convert non-finite values (NA or inf) to integer


In [57]:
resumen_nans_sbpro = resumen_nans_df(sbpro)
resumen_nans_sbpro

,n_nans,n_no_nans,porcentaje_nans
punt_global,253606.0,1014827.0,19.993646
estu_prgm_academico,27516.0,1240917.0,2.169291
inst_nombre_institucion,27516.0,1240917.0,2.169291
estu_prgm_departamento,27516.0,1240917.0,2.169291
estu_prgm_municipio,27516.0,1240917.0,2.169291
estu_prgm_codmunicipio,27516.0,1240917.0,2.169291
estu_nivel_prgm_academico,27516.0,1240917.0,2.169291
estu_inst_departamento,27516.0,1240917.0,2.169291
inst_caracter_academico,27516.0,1240917.0,2.169291
estu_inst_codmunicipio,27516.0,1240917.0,2.169291


### Agregar la columan punt_global para el 2021

In [58]:
# Aplicar solo a filas con periodo // 10 == 2021
mask = (sbpro["periodo"] // 10 == 2021)
sbpro.loc[mask, "punt_global"] = sbpro.loc[mask, [
    "mod_ingles_punt",
    "mod_competen_ciudada_punt",
    "mod_lectura_critica_punt",
    "mod_razona_cuantitat_punt",
    "mod_comuni_escrita_punt"
]].fillna(0).mean(axis=1).round()

### Exportar los datos

In [59]:
from datetime import datetime

# Generate today's date string in DDMMYYYY format
fecha_hoy = datetime.today().strftime('%d%m%Y')
#Guardar el dataframe como un archivo csv en la carpeta SABERPRO_cleaned
sbpro.to_csv(f"../../data/SABERPRO_cleaned/base_sbpro_nacional_corte_{fecha_hoy}.csv", index=False)

In [60]:
municipios = pd.read_csv("../../data/Municipios_cleaned/municipios.csv")
sbpro_bogota_region = sbpro[sbpro["estu_prgm_codmunicipio"].isin(municipios["codigo_dane_municipio"])]

sbpro_bogota_region.to_csv(f"../../data/SABERPRO_cleaned/base_sbpro_bogota_region_corte_{fecha_hoy}.csv", index=False)